In [1]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/p/project/chhb19/gaertner2/lkf_detection/functions/')
from statistics_functions import *
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = '/p/project/chhb19/gaertner2/data/awicm_cvmix/'

In [3]:
LKFs = np.load(path + 'LKFs.npy', allow_pickle=True)[0]

In [4]:
[years, lkfs, paths, paths_all] = np.load(path + 'data_lkfs_paths.npy', allow_pickle=True)

In [5]:
for year in years:
    
    yr = np.where(years==year)[0][0]
    for day in range(365):

        eps_d, div_d, shr_d, vor_d = [], [], [], []
        for lkf in lkfs[yr][day]:

            div_d += np.mean(lkf[:,4]),
            shr_d += np.mean(lkf[:,5]),
            vor_d += np.mean(lkf[:,6]),

        eps_d = np.sqrt(np.array(div_d)**2+np.array(shr_d)**2)

        LKFs[f'y{year}'][f'd{day}']['deformation'] = eps_d
        LKFs[f'y{year}'][f'd{day}']['divergence']  = div_d
        LKFs[f'y{year}'][f'd{day}']['shear']       = shr_d
        LKFs[f'y{year}'][f'd{day}']['vorticity']   = vor_d

In [6]:
for year in years:
    LKFs[f'y{year}']['d0']['lead or ridge'] = np.where(LKFs[f'y{year}']['d0']['divergence']<0, 1, 2)
    
    yr = np.where(years==year)[0][0]
    for day in range(1,365):
        
        LKFs[f'y{year}'][f'd{day}']['lead or ridge'] = np.where(LKFs[f'y{year}'][f'd{day}']['divergence']<0, 1, 2)
        for lkf in range(len(lkfs[yr][day])):
            
            for lkf_path in paths_all[0][0]:
                if np.size(lkf_path)>1:
                    if lkf_path[1] == lkf:
                        LKFs[f'y{year}'][f'd{day}'].loc[lkf, 'lead or ridge'] = 3
                        break

In [7]:
np.save(path + 'LKFs', [LKFs])

1 = lead, 2 = ridge, 3 = can't tell (this is for longer lived LKFs: divergence<0 => lead only works on the first day of the LKF)

In [8]:
LKFs['y2013']['d176']

,length,lifetime,deformation,divergence,shear,vorticity,lead or ridge,growth rate,curvature
0,141.968791,1,0.501648,0.096004,0.492376,-0.113951,3,NaN,0.113380
1,46.227827,1,0.510144,-0.325587,0.392733,-0.145198,1,NaN,0.075819
2,52.362472,2,0.546260,0.322290,0.441055,0.075979,3,0.000000,0.073790
3,51.309933,1,1.584526,0.239210,1.566366,0.087489,2,NaN,0.054791
4,54.903525,1,0.399023,-0.273015,0.291001,0.074604,3,NaN,0.075228
...,...,...,...,...,...,...,...,...,...
826,74.360000,1,0.123426,0.048350,0.113562,0.039993,2,NaN,0.050362
827,85.576751,1,0.278532,0.201956,0.191818,0.004262,2,NaN,0.069463
828,72.999179,1,0.166901,-0.111329,0.124345,0.014354,1,NaN,0.041829
829,41.889978,1,0.161923,-0.111293,0.117613,-0.037202,1,NaN,0.073790
